In [4]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import json

In [17]:
baseUrl = 'https://www.nfl.com'
teamAbbr = pd.Series(["SF","CHI","CIN","BUF","DEN","CLE","TB","ARI","LAC","KC","IND","DAL","MIA","PHI","ATL","NYG",
                     "JAX","NYJ","DET","GB","CAR","NE","OAK","LAR","BAL","WAS","NO","SEA","PIT","HOU","TEN","MIN"])
teamLocation = pd.Series(["San Francisco","Chicago","Cincinatti","Buffalo","Denver","Cleveland","Tampa Bay","Arizona",
                         "Los Angeles","Kansas City","Indianapolis","Dallas","Miami","Philadelphia","Atlanta","New York",
                         "Jacksonville","New York","Detroit","Green Bay","Carolina","New England","Oakland","Los Angeles",
                         "Baltimore","Washington","New Orleans","Seattle","Pittsburgh","Houston","Tennessee","Minnesota"])

In [6]:
playerPayload = {"type":"/stats","subject":"player-stats","category":"category",
                 "statisticCategory":"rushing","season":"2019",
                 "seasonType":"REG",
                 "list":"all",
                 "specificCategory":"rushingYards",
                 "order":"DESC"}

teamPayload = {"type":"/stats","subject":"team-stats","category":"offense",
               "statisticCategory":"passing",
               "season":"2019",
               "seasonType":"REG",
               "list":"all",
               "specificCategory":"",
               "order":""}

schedulePayload = {"type":"/schedules",
                  "season":"2019",
                  "seasonType":"REG"}


In [10]:
payload = schedulePayload

url = 'https://www.nfl.com'
for key in payload:
    if payload[key]:
        url += payload[key]
        url += '/'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')

In [54]:
season = soup.find_all('select', {"class":"d3-o-dropdown"})[1].find_all("option")
seasonWeeks = []
for week in season:
    seasonWeeks.append(week.get("value"))
current = soup.find_all('select', {"class":"d3-o-dropdown"})[1].find('option', {"selected":True}).get("value")
print(current)
seasonWeeks
for week in seasonWeeks:
    time.sleep(1)
    response = requests.get(baseUrl+week)
    print("Response:", response.status_code,response.url)

/schedules/2019/REG1/
Response: 200 https://www.nfl.com/schedules/2019/PRE0/
Response: 200 https://www.nfl.com/schedules/2019/PRE1/
Response: 200 https://www.nfl.com/schedules/2019/PRE2/
Response: 200 https://www.nfl.com/schedules/2019/PRE3/
Response: 200 https://www.nfl.com/schedules/2019/PRE4/
Response: 200 https://www.nfl.com/schedules/2019/REG1/
Response: 200 https://www.nfl.com/schedules/2019/REG2/
Response: 200 https://www.nfl.com/schedules/2019/REG3/
Response: 200 https://www.nfl.com/schedules/2019/REG4/


KeyboardInterrupt: 

In [12]:
tableRows = []
pagingText = True


while pagingText:
    time.sleep(2)
    response = requests.get(url)
    print("url:",response.url)
    soup = BeautifulSoup(response.text,'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')

    for row in rows:
        cols = row.find_all('td')
        if cols:
            cols = [ele.text.strip() for ele in cols]
            tableRows.append([ele for ele in cols if ele]) # Get rid of empty values
    pagingText = soup.find("a", {"class": "nfl-o-table-pagination__next"})
    if pagingText:
        url = baseUrl + pagingText.get('href')

url: https://www.nfl.com/schedules/2019/REG/


AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
resultsDf = pd.DataFrame(tableRows, columns=columnHeader)
# remove \n whitespace from team names
find = re.compile(r"^(\n*).*")
resultsDf['Team'] = resultsDf['Team'].apply(lambda x: re.search(find,x).group())
pd.DataFrame.head(resultsDf)

In [18]:
def getStats(url, payload): 
    tableRows = []
    pagingText = True
    
    for key in payload:
        if payload[key]:
            url += payload[key]
            url += '/'
    response = requests.get(url)
    
    # get column names
    soup = BeautifulSoup(response.text,'html.parser')
    columnHeader = []
    header = soup.find('table').find_all('th')
    for col in header:
        columnHeader.append(col.text)
    columnHeader = [c.strip('\n') for c in columnHeader]

    # search through pages and append to tableRows
    while pagingText:
        time.sleep(2)
        response = requests.get(url)
        print("url:",response.url)
        soup = BeautifulSoup(response.text,'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr')

        for row in rows:
            cols = row.find_all('td')
            if cols:
                cols = [ele.text.strip() for ele in cols]
                cols = [ele.rstrip('\n') for ele in cols]
                tableRows.append([ele for ele in cols if ele]) # Get rid of empty values
        pagingText = soup.find("a", {"class": "nfl-o-table-pagination__next"})
        if pagingText:
            url = baseUrl + pagingText.get('href')
    
    resultsDf = pd.DataFrame(tableRows, columns=columnHeader)
    find = re.compile(r"^(\n*).*")
    resultsDf['Team'] = resultsDf['Team'].apply(lambda x: re.search(find,x).group())
    
    # add teamId to table
    resultsDf = resultsDf.sort_values(by=['Team']).reset_index(drop=True)
    resultsDf['teamId'] = teamAbbr
    resultsDf['teamLoc'] = teamLocation
    return resultsDf

In [20]:
teamPayload = {"stats":"/stats","subject":"team-stats",
               "category":"offense",
               "statisticCategory":"passing",
               "season":"2019",
               "seasonType":"REG",
               "list":"all",
               "specificCategory":"",
               "order":""}
df1 = getStats(baseUrl,teamPayload)
pd.DataFrame.head(df1)

url: https://www.nfl.com/stats/team-stats/offense/passing/2019/reg/all


,Team,Att,Cmp,Cmp %,Yds/Att,Pass Yds,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY,teamId,teamLoc
0,49ers,478,331,69.2,8.4,4029,28,13,103.1,195,40.8,62,8,75T,36,237,SF,San Francisco
1,Bears,580,371,64,6.2,3573,20,12,83.9,178,30.7,39,2,53,45,282,CHI,Chicago
2,Bengals,616,356,57.8,6.5,3994,18,16,76.2,191,31,48,7,66T,48,342,CIN,Cincinatti
3,Bills,513,299,58.3,6.8,3476,21,12,82.8,162,31.6,54,8,53T,40,247,BUF,Buffalo
4,Broncos,504,312,61.9,6.7,3401,16,10,84.1,162,32.1,46,10,75T,41,286,DEN,Denver


In [25]:
teamDf = df1[['teamId','teamLoc','Team']]
pd.DataFrame.head(teamDf,20)

teamsJson = teamDf.to_json(orient='records')

with open("teams.json","w") as outfile:
    outfile.write(teamsJson)

In [1]:
def getMatchups(url,payload):
    for key in payload:
        if payload[key]:
            url += payload[key]
            url += '/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    
    matchups = []
    columnHeader = ['matchupId', 'team1', 'team2', 'team1Score','team2Score','weekType','weekNum']
    season = soup.find_all('select', {"class":"d3-o-dropdown"})[1].find_all("option")
    seasonWeeks = []
    for week in season:
        seasonWeeks.append(week.get("value"))
    for week in seasonWeeks:
        time.sleep(2)
        url = baseUrl + week
        response = requests.get(url)
        print(url)
        soup = BeautifulSoup(response.text,'html.parser')
        weekName = week.rsplit('/', 2)[-2]
        weekType = re.findall("[a-zA-Z]+", weekName)[0]
        weekNum = re.findall(r'\d+', weekName)[0]
        games = soup.find_all('a', {"class": "nfl-c-matchup-strip__game"})
        index = 0
        for game in games:
            gameSoup = game.find_all('span', {"class":"nfl-c-matchup-strip__team-abbreviation"})
            matchup = []
            matchup.append(str(index))
            for team in gameSoup:
                matchup.append(re.sub(r'[^A-Za-z]', '', team.get_text()))
            scoreSoup = game.find_all('div', {'class':"nfl-c-matchup-strip__team-score"})
            if scoreSoup:
                for team in scoreSoup:
                    matchup.append(team.get_text())
            else:
                matchup.append("")
                matchup.append("")
            matchup.append(weekType)
            matchup.append(weekNum)
            matchups.append(matchup)
            index += 1
    matchupsDf = pd.DataFrame(matchups,columns=columnHeader)
    return matchupsDf

In [7]:
schedulePayload = {"type":"/schedules",
                  "season":"2019",
                  "seasonType":"REG"}
df2 = getMatchups(baseUrl,schedulePayload)

#matchupsDf.loc[(matchupsDf['weekType']=='REG') & (matchupsDf['weekNum']=='1')]

https://www.nfl.com/schedules/2019/PRE0/
https://www.nfl.com/schedules/2019/PRE1/
https://www.nfl.com/schedules/2019/PRE2/
https://www.nfl.com/schedules/2019/PRE3/
https://www.nfl.com/schedules/2019/PRE4/
https://www.nfl.com/schedules/2019/REG1/
https://www.nfl.com/schedules/2019/REG2/
https://www.nfl.com/schedules/2019/REG3/
https://www.nfl.com/schedules/2019/REG4/
https://www.nfl.com/schedules/2019/REG5/
https://www.nfl.com/schedules/2019/REG6/
https://www.nfl.com/schedules/2019/REG7/
https://www.nfl.com/schedules/2019/REG8/
https://www.nfl.com/schedules/2019/REG9/
https://www.nfl.com/schedules/2019/REG10/
https://www.nfl.com/schedules/2019/REG11/
https://www.nfl.com/schedules/2019/REG12/
https://www.nfl.com/schedules/2019/REG13/
https://www.nfl.com/schedules/2019/REG14/
https://www.nfl.com/schedules/2019/REG15/
https://www.nfl.com/schedules/2019/REG16/
https://www.nfl.com/schedules/2019/REG17/
https://www.nfl.com/schedules/2019/POST1/
https://www.nfl.com/schedules/2019/POST2/
https:

In [60]:
df2

,matchupId,team1,team2,team1Score,team2Score,weekType,weekNum
0,0,DEN,ATL,14,10,PRE,0
1,0,NYJ,NYG,22,31,PRE,1
2,1,IND,BUF,16,24,PRE,1
3,2,TEN,PHI,27,10,PRE,1
4,3,ATL,MIA,27,34,PRE,1
...,...,...,...,...,...,...,...
328,3,SEA,GB,23,28,POST,2
329,0,TEN,KC,24,35,POST,3
330,1,GB,SF,20,37,POST,3
331,0,NFC,AFC,33,38,PRO,1


In [11]:
matchupsJson = df2.to_json(orient='records')

with open("matchups.json","w") as outfile:
    outfile.write(matchupsJson)

In [65]:
weekTypeD = {"PRE":"Pre-Season", "REG":"Regular Season", "POST": "Post-Season"}


def getWeeks(url,payload):
    for key in payload:
        if payload[key]:
            url += payload[key]
            url += '/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    columnHeader = ['weekType','weekNum','weekTypeF','weekF']
    matchups = []
    season = soup.find_all('select', {"class":"d3-o-dropdown"})[1].find_all("option")
    seasonWeeks = []
    for week in season:
        seasonWeeks.append(week.get("value"))
    for week in seasonWeeks:
        url = baseUrl + week
        weekName = week.rsplit('/', 2)[-2]
        weekType = re.findall("[a-zA-Z]+", weekName)[0]
        weekNum = re.findall(r'\d+', weekName)[0]
        if weekType in weekTypeD.keys():
            weekTypeF = weekTypeD[weekType]
            weekF = weekTypeF + " Week " + weekNum
            matchup = []
            matchup.append(weekType)
            matchup.append(weekNum)
            matchup.append(weekTypeF)
            matchup.append(weekF)
            matchups.append(matchup)
    weeksDf = pd.DataFrame(matchups,columns=columnHeader)
    return weeksDf

In [66]:
schedulePayload = {"type":"/schedules",
                  "season":"2018",
                  "seasonType":"REG"}
df3 = getWeeks(baseUrl,schedulePayload)
df3
weeksJson = df3.to_json(orient='records')

with open("weeks.json","w") as outfile:
    outfile.write(weeksJson)